# Load Libraries

In [2]:
import pandas as pd
from pylab import *
import seaborn as sns
sns.set(style="whitegrid")
import csv
import gc
import scipy
from scipy.stats.stats import pearsonr, spearmanr
import time
from tqdm import *

pd.set_option('display.max_columns', None)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['linalg', '__version__', 'info', 'fft', 'power', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
time_start_notebook = time.time()

# Define import and export paths

In [ ]:
import_path = '/Users/g.triulzi/Dropbox (MIT)/Paper Predicting K with Patents/Data/Input/'
export_path = '/Users/g.triulzi/Dropbox (MIT)/Paper Predicting K with Patents/Data/Output/'

# Import Data

In [5]:
ti = time.time() # 6.9

# Read file with list of patents for each domain
PATENT_SET_DOMAINS = pd.read_csv(import_path+'Domains_patent_info.csv')
# Read file with information on all utility patents granted by the USPTO between 1975 and 2015
PATENT_INFO = pd.read_csv(import_path+'All_patents_info.csv')
PATENT_INFO.drop_duplicates('patent_number', keep = 'first', inplace=True)
idx_letters = PATENT_INFO['mainclass_id'].astype(str).str.contains('a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|/|;|-|nan', na=False, case=False)
PATENT_INFO['mainclass_id'][~idx_letters] = PATENT_INFO['mainclass_id'][~idx_letters].astype(int).astype(str)
PATENT_INFO['mainclass_id'] = PATENT_INFO['mainclass_id'].astype(str)
# Read file with information on all citations between utility patents granted by the USPTO between 1975 and 2015
CITATION_INFO = pd.read_csv(import_path+'CITATION_INFO_no_neg_citlag.csv')
del CITATION_INFO['Unnamed: 0']
gc.collect()

tf = time.time()
(tf-ti)/60

C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


2.6172067205111187

In [6]:
ti = time.time() #1.13

# create a series with 'patent_number' as index and ID from zero to N as values. 
# We will then use it to translate the edgelist from patent numbers to patent IDs. We will use 'map' for that
PATENT_INFO['ID'] = range(len(PATENT_INFO['patent_number']))
series_patent_ids = PATENT_INFO[['ID','patent_number']].set_index('patent_number').ix[:,0] 

# Now map patent numbers to IDs in our edgelist
CITATION_INFO['citing_numerical_id'] = CITATION_INFO['citing_id'].astype(int).map(series_patent_ids) # see: http://stackoverflow.com/questions/25653652/how-to-replace-efficiently-values-on-a-pandas-dataframe
CITATION_INFO['cited_numerical_id'] = CITATION_INFO['cited_id'].astype(int).map(series_patent_ids)

# add information on USPC main class of citing and cited nodes to citation data
CITATION_INFO['citing_numerical_id'] = CITATION_INFO['citing_numerical_id'].astype(int)
CITATION_INFO['cited_numerical_id'] = CITATION_INFO['cited_numerical_id'].astype(int)
CITATION_INFO['USPC_mainclass_citing'] = array(pd.Series(PATENT_INFO['mainclass_id'])[array(CITATION_INFO['citing_numerical_id'])])
CITATION_INFO['USPC_mainclass_cited'] = array(pd.Series(PATENT_INFO['mainclass_id'])[array(CITATION_INFO['cited_numerical_id'])])

# add information on IPC main class of citing and cited nodes to citation data
CITATION_INFO['IPC_mainclass_citing'] = array(pd.Series(PATENT_INFO['IPC4'])[array(CITATION_INFO['citing_numerical_id'])])
CITATION_INFO['IPC_mainclass_cited'] = array(pd.Series(PATENT_INFO['IPC4'])[array(CITATION_INFO['cited_numerical_id'])])

tf = time.time()
(tf-ti)/60

C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


0.8315591653188069

In [7]:
CITATION_INFO['same_USPC_class'] = CITATION_INFO['USPC_mainclass_citing'].astype(str) == CITATION_INFO['USPC_mainclass_cited'].astype(str)
CITATION_INFO['same_IPC_class'] = CITATION_INFO['IPC_mainclass_citing'].astype(str) == CITATION_INFO['IPC_mainclass_cited'].astype(str)

# Compute observed number of citations to given age within and between classes for each patent

In [8]:
ti = time.time() #5.8

DF_patents_citations_age_within_class = (CITATION_INFO[['citing_id','CITLAG_filing_year','citation_id']]
                                         [(CITATION_INFO['same_USPC_class']==True) & 
                                          (CITATION_INFO['CITLAG_filing_year']<=40)]
                                         .groupby(['citing_id','CITLAG_filing_year']).agg(['count']))

DF_patents_citations_age_between_class = (CITATION_INFO[['citing_id','CITLAG_filing_year','citation_id']]
                                         [(CITATION_INFO['same_USPC_class']==False) & 
                                          (CITATION_INFO['CITLAG_filing_year']<=40)]
                                         .groupby(['citing_id','CITLAG_filing_year']).agg(['count']))

DF_patents_citations_age_total = (CITATION_INFO[['citing_id','CITLAG_filing_year','citation_id']]
                                         [CITATION_INFO['CITLAG_filing_year']<=40]
                                  .groupby(['citing_id','CITLAG_filing_year']).agg(['count']))

tf = time.time()
(tf-ti)/60

1.3280476013819376

In [9]:
del CITATION_INFO
gc.collect()

20

In [10]:
# Move pandas data from index to column after groupby
# http://stackoverflow.com/questions/21767900/how-to-move-pandas-data-from-index-to-column-after-multiple-groupby
DF_patents_citations_age_within_class = DF_patents_citations_age_within_class['citation_id'].reset_index()
DF_patents_citations_age_between_class = DF_patents_citations_age_between_class['citation_id'].reset_index()
DF_patents_citations_age_total = DF_patents_citations_age_total['citation_id'].reset_index()

In [11]:
# Rename columns of groupby output to clarify if the citation count refers to within or between classes
DF_patents_citations_age_total.rename(columns={"CITLAG_filing_year": "age","count": "citation_count_age"},inplace=True)
DF_patents_citations_age_within_class.rename(columns={"CITLAG_filing_year": "age","count": "within_class_citation_count_age"},inplace=True)
DF_patents_citations_age_between_class.rename(columns={"CITLAG_filing_year": "age","count": "between_class_citation_count_age"},inplace=True)

# Merge with within class citations counts
DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         DF_patents_citations_age_within_class, 
                             how = 'left', left_on = ['citing_id','age'], right_on = ['citing_id','age']))

DF_patents_citations_age_total['within_class_citation_count_age'][isnan(DF_patents_citations_age_total['within_class_citation_count_age'])]=0.0

# Merge with between class citations counts
DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         DF_patents_citations_age_between_class, 
                             how = 'left', left_on = ['citing_id','age'], right_on = ['citing_id','age']))

DF_patents_citations_age_total['between_class_citation_count_age'][isnan(DF_patents_citations_age_total['between_class_citation_count_age'])]=0.0


C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
DF_patents_citations_age_total['citing_id'] = DF_patents_citations_age_total['citing_id'].astype(int)
PATENT_INFO['patent_number'] = PATENT_INFO['patent_number'].astype(int)

DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         PATENT_INFO[['patent_number','mainclass_id']], 
                             how = 'left', left_on = ['citing_id'], right_on = ['patent_number']))

del DF_patents_citations_age_total['patent_number']
gc.collect()

20

In [13]:
# Add columns with total number of citations made by each patent to SAME class
DF_sum_within_class_citations_by_patent = (DF_patents_citations_age_total[['citing_id','within_class_citation_count_age']]
                                           .groupby(['citing_id']).agg(['sum']))

DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         DF_sum_within_class_citations_by_patent['within_class_citation_count_age'], 
                             how = 'left', left_on = ['citing_id'], 
                                         right_on = array(DF_sum_within_class_citations_by_patent['within_class_citation_count_age']['sum'].index)))

DF_patents_citations_age_total.rename(columns={"sum": "total_within_class_citation_count"},inplace=True)

# Add columns with total number of citations made by each patent to OTHER classes
DF_sum_between_class_citations_by_patent = (DF_patents_citations_age_total[['citing_id','between_class_citation_count_age']]
                                           .groupby(['citing_id']).agg(['sum']))

DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         DF_sum_between_class_citations_by_patent['between_class_citation_count_age'], 
                             how = 'left', left_on = ['citing_id'], 
                                         right_on = array(DF_sum_between_class_citations_by_patent['between_class_citation_count_age']['sum'].index)))

DF_patents_citations_age_total.rename(columns={"sum": "total_between_class_citation_count"},inplace=True)

In [14]:
# Add columns with patent's filing year
DF_patents_citations_age_total=(pd.merge(DF_patents_citations_age_total,
                                         PATENT_INFO[['patent_number','filing_year']], 
                             how = 'left', left_on = 'citing_id', 
                                         right_on = 'patent_number'))

del DF_patents_citations_age_total['patent_number']
gc.collect()

20

In [15]:
DF_patents_citations_age_total.rename(columns={"mainclass_id": "Class", "age":"Age"},inplace=True)

In [16]:
DF_patents_citations_age_total[:8]

,citing_id,Age,citation_count_age,within_class_citation_count_age,between_class_citation_count_age,Class,total_within_class_citation_count,total_between_class_citation_count,filing_year
0,3930394,26.0,1,0.0,1.0,72,0.0,1.0,1974.0
1,3930419,28.0,1,0.0,1.0,74,0.0,1.0,1973.0
2,3930430,15.0,1,0.0,1.0,84,0.0,1.0,1974.0
3,3930447,1.0,1,0.0,1.0,101,1.0,1.0,1974.0
4,3930447,17.0,1,1.0,0.0,101,1.0,1.0,1974.0
5,3930505,7.0,1,0.0,1.0,604,0.0,1.0,1974.0
6,3930572,1.0,1,0.0,1.0,198,0.0,1.0,1974.0
7,3930653,4.0,1,0.0,1.0,369,0.0,1.0,1975.0


# Compute number of patents of a given age (up to 40 years old) in each USPC class at each point in time

In [17]:
# Read file with data provided by Hyejin Youn on class size since early days of USPTO
HISTORIC_USPTO = pd.read_csv('/Users/g.triulzi/Dropbox (MIT)/USPTO Database 2015/Hyejin_Youn_Database_Classes_sorted.csv')

In [19]:
# I need to harmonize the way class names with number lower than 100 are recorded between different dataframes
list_classes_to_edit = (['001', '002', '004', '005', '007', '008', '012', '014', '015',
       '016', '019', '023', '024', '026', '027', '028', '029', '030',
       '033', '034', '036', '037', '038', '040', '042', '043', '044',
       '047', '048', '049', '051', '052', '053', '054', '055', '056',
       '057', '059', '060', '062', '063', '065', '066', '068', '069',
       '070', '071', '072', '073', '074', '075', '076', '079', '081',
       '082', '083', '084', '086', '087', '089', '091', '092', '095',
       '096', '099'])

for x in tqdm(list_classes_to_edit):
    HISTORIC_USPTO['Class'][HISTORIC_USPTO['Class'].astype(str).str.contains(x)]=int(x)

  0%|                                                                                           | 0/65 [00:00<?, ?it/s]C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [14:18<00:00, 13.21s/it]


In [20]:
# Take into account primary class only for utility patents only
HISTORIC_USPTO_utility_primaryclass = HISTORIC_USPTO[(HISTORIC_USPTO['Pat_Type']=='U') & (HISTORIC_USPTO['Primary']==1)]

DF_class_size_by_age = pd.DataFrame()
year_list = unique(HISTORIC_USPTO_utility_primaryclass['Appyear'])
for year in tqdm(year_list):
    DATA = HISTORIC_USPTO_utility_primaryclass[['Patent','Class','Appyear']][(HISTORIC_USPTO_utility_primaryclass['Appyear']>=year-40) & (HISTORIC_USPTO_utility_primaryclass['Appyear']<=year)]
    DATA['Age']=year-DATA['Appyear']
    
    # Compute number of patents of a given age (smaller or equal than 40) in each class in that year
    DATA_grouped = DATA.groupby(['Class','Age']).agg(['count'])
    DATA_grouped = DATA_grouped['Patent'].reset_index()
    DATA_grouped['year_of_observation']=year
    DATA_grouped.rename(columns={"count": "patcount_class_age_year"}, inplace=True)
    
    # Compute number of patents of a given age (smaller or equal than 40) in ANY class in that year
    DATA_grouped_all_classes = DATA.groupby(['Age']).agg(['count'])
    DATA_grouped=(pd.merge(DATA_grouped, DATA_grouped_all_classes['Patent'], 
                             how = 'left', left_on = 'Age', right_on = array(DATA_grouped_all_classes['Patent']['count'].index)))
    DATA_grouped.rename(columns={"count": "patcount_all_classes_age_year"}, inplace=True)
    
    # Compute number of patents of a given age (smaller or equal than 40) in all other classes in that year
    DATA_grouped['patcount_other_classes_age_year'] = DATA_grouped['patcount_all_classes_age_year'] - DATA_grouped['patcount_class_age_year']
    
    # Compute total number of patents in class in the last 40 years in that year
    DATA_grouped_total_in_class = DATA[['Patent', 'Class']].groupby(['Class']).agg(['count'])
    DATA_grouped=(pd.merge(DATA_grouped, DATA_grouped_total_in_class['Patent'], 
                             how = 'left', left_on = 'Class', right_on = array(DATA_grouped_total_in_class['Patent']['count'].index)))
    DATA_grouped.rename(columns={"count": "patcount_class_total_below40_year"}, inplace=True)
    
    # Compute total number of patents in ANY class in the last 40 years in that year
    DATA_grouped['patcount_total_below40_year'] = shape(DATA)[0]
    
    # Compute total number of patents in ANY OTHER class in the last 40 years in that year
    DATA_grouped['patcount_other_classes_total_below40_year'] = DATA_grouped['patcount_total_below40_year'] - DATA_grouped['patcount_class_total_below40_year']
    
    # Append data for given year of observation to the complete DataFrame
    DF_class_size_by_age = DF_class_size_by_age.append(DATA_grouped)
    
    del DATA
    del DATA_grouped
    
gc.collect()

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:57<00:00,  1.52it/s]


0

In [21]:
del HISTORIC_USPTO
del HISTORIC_USPTO_utility_primaryclass
gc.collect()

20

# Merge DF_patents_citations_age_total and DF_class_size_by_age

In [23]:
# Now create a dataframe with each citing_id repeated 41 times, once per each potential age of the cited patents
list_citing_ids = unique(DF_patents_citations_age_total['citing_id'])
list_ages = list(range(41))

DF_final = pd.DataFrame({"citing_id":tile(list_citing_ids,len(list_ages)), "Age":tile(list_ages,len(list_citing_ids))})
DF_final = DF_final.sort_values(['citing_id','Age'])

In [24]:
# Merge the newly created dataframe DF_final with DF_patents_citations_age_total and DF_class_size_by_age
DF_final['citing_id'] = DF_final['citing_id'].astype(str)
DF_patents_citations_age_total['citing_id'] = DF_patents_citations_age_total['citing_id'].astype(str)

DF_final=(pd.merge(DF_final, DF_patents_citations_age_total,
                         how = 'left', left_on = ['citing_id','Age'], right_on = ['citing_id','Age']))

# Add the complete information on total_within_class_citation_count and total_between_class_citation_count (we first need to delete the old
# columns because they do not contain the information for each patent -age pair)
del DF_final['total_within_class_citation_count']
del DF_final['total_between_class_citation_count']
# Add columns with total number of citations made by each patent to SAME class
DF_final.citing_id = DF_final.citing_id.astype(int)
DF_final=(pd.merge(DF_final,DF_sum_within_class_citations_by_patent['within_class_citation_count_age'], 
                             how = 'left', left_on = ['citing_id'], 
                                         right_on = array(DF_sum_within_class_citations_by_patent['within_class_citation_count_age']['sum'].index)))

DF_final.rename(columns={"sum": "total_within_class_citation_count"},inplace=True)

# Add columns with total number of citations made by each patent to OTHER classes
DF_final=(pd.merge(DF_final,DF_sum_between_class_citations_by_patent['between_class_citation_count_age'], 
                             how = 'left', left_on = ['citing_id'], 
                                         right_on = array(DF_sum_between_class_citations_by_patent['between_class_citation_count_age']['sum'].index)))

DF_final.rename(columns={"sum": "total_between_class_citation_count"},inplace=True)

# Add the complete information on patent class and filing year (we first need to delete the old columns because they do not 
# contain the information for each patent -age pair)
del DF_final['Class']
del DF_final['filing_year']
PATENT_INFO['patent_number'] = PATENT_INFO['patent_number'].astype(int)

DF_final=(pd.merge(DF_final, PATENT_INFO[['patent_number','mainclass_id','filing_year']],
                         how = 'left', left_on = ['citing_id'], right_on = ['patent_number']))
DF_final.rename(columns={"mainclass_id": "Class"}, inplace=True)
del DF_final['patent_number']

# Now merge with DF_class_size_by_age
DF_final['Class'] = DF_final['Class'].astype(str)
DF_class_size_by_age['Class'] = DF_class_size_by_age['Class'].astype(str)

DF_final=(pd.merge(DF_final, DF_class_size_by_age, 
                         how = 'left', left_on = ['Class','Age','filing_year'], right_on = ['Class','Age','year_of_observation']))

# Replace NaN with zero in columns citation_count_age, within_class_citation_count_age and between_class_citation_count_age
DF_final['citation_count_age'][isnan(DF_final['citation_count_age'])]= 0.0
DF_final['within_class_citation_count_age'][isnan(DF_final['within_class_citation_count_age'])]= 0.0
DF_final['between_class_citation_count_age'][isnan(DF_final['between_class_citation_count_age'])]= 0.0

C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\g.triulzi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Compute Expected Value, STD and Z-score of number of citations within and between classes by age for each patent

In [26]:
# Compute EXPEXTED VALUES
DF_final['EV_num_cit_within_class_age'] = (DF_final['total_within_class_citation_count'] * 
                                                                 DF_final['patcount_class_age_year'] /
                                                                 DF_final['patcount_class_total_below40_year'])

DF_final['EV_num_cit_between_class_age'] = (DF_final['total_between_class_citation_count'] * 
                                                                 DF_final['patcount_other_classes_age_year'] /
                                                                 DF_final['patcount_other_classes_total_below40_year'])

DF_final['EV_num_cit_total_age'] = (DF_final['EV_num_cit_within_class_age'] + 
                                                          DF_final['EV_num_cit_between_class_age'])

In [27]:
# Compute STANDARD DEVIATIONS
DF_final['STD_num_cit_within_class_age'] = (DF_final['EV_num_cit_within_class_age'] *
                                           (1 - (DF_final['patcount_class_age_year'] / DF_final['patcount_class_total_below40_year'])) *
                                           (DF_final['patcount_class_total_below40_year']-DF_final['total_within_class_citation_count'])/
                                            (DF_final['patcount_class_total_below40_year']-1))

DF_final['STD_num_cit_between_class_age'] = (DF_final['EV_num_cit_between_class_age'] *
                                           (1 - (DF_final['patcount_other_classes_age_year'] / DF_final['patcount_other_classes_total_below40_year'])) *
                                           (DF_final['patcount_other_classes_total_below40_year']-DF_final['total_between_class_citation_count'])/
                                            (DF_final['patcount_other_classes_total_below40_year']-1))

DF_final['STD_num_cit_total_age'] = (DF_final['STD_num_cit_within_class_age'] + 
                                                          DF_final['STD_num_cit_between_class_age'])

In [28]:
# Compute Z-SCORES
DF_final['Z_num_cit_within_class_age'] = ((DF_final['within_class_citation_count_age'] - 
                                                                 DF_final['EV_num_cit_within_class_age']) /
                                                                 DF_final['STD_num_cit_within_class_age'])

DF_final['Z_num_cit_between_class_age'] = ((DF_final['between_class_citation_count_age'] - 
                                                                 DF_final['EV_num_cit_between_class_age']) /
                                                                 DF_final['STD_num_cit_between_class_age'])

DF_final['Z_num_cit_total_age'] = ((DF_final['citation_count_age'] - 
                                                                 DF_final['EV_num_cit_total_age']) /
                                                                 DF_final['STD_num_cit_total_age'])

# Free memory

In [30]:
del DATA_grouped_all_classes
del DATA_grouped_total_in_class
del DF_class_size_by_age
del DF_patents_citations_age_between_class
del DF_patents_citations_age_total
del DF_patents_citations_age_within_class
del DF_sum_between_class_citations_by_patent
del DF_sum_within_class_citations_by_patent
gc.collect()

198

In [43]:
del list_citing_ids
del PATENT_INFO
del series_patent_ids
del idx_letters
gc.collect()

40

# Compute Normalized Knowledge Obsolescence Index at the domain level

In [44]:
PATENT_SET_DOMAINS['patent_id'] = PATENT_SET_DOMAINS['patent_id'].astype(str)
DF_final['citing_id'] = DF_final['citing_id'].astype(str)

In [ ]:
DF_final=(pd.merge(DF_final,PATENT_SET_DOMAINS[['patent_id','Domain']],
                         how = 'left', left_on = ['citing_id'], right_on = ['patent_id']))

In [ ]:
del DF_final['patent_id']
del PATENT_SET_DOMAINS
gc.collect()

In [ ]:
DZ_years_domains = pd.DataFrame()
list_domains = DF_final['Domain'].unique()
list_domains = list_domains[list_domains.astype(str)!='nan']
for domain in tqdm(list_domains):
    year_list_domain = list(range(1975,2014))
    for year in year_list_domain:
        domain_name = domain
        DX = DF_final[(DF_final['Domain']==domain_name) & (DF_final['filing_year']<=year)]
        DX_grouped = DX[['Age','citation_count_age','EV_num_cit_total_age','STD_num_cit_total_age']].groupby('Age').agg('sum')
        DX_grouped = DX_grouped.reset_index()
        DX_grouped['Zscore_citations_age'] = (DX_grouped['citation_count_age']-DX_grouped['EV_num_cit_total_age'])/DX_grouped['STD_num_cit_total_age']
        
        # store z-score of observed PDF at different ages
        Z0 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==0]
        Z1 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==1]
        Z2 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==2]
        Z3 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==3]
        Z4 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==4]
        Z5 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==5]
        Z10 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==10]
        Z20 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==20]
        Z30 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==30]
        Z40 = DX_grouped['Zscore_citations_age'][DX_grouped['Age']==40]
        
        # compute z-score of observed CDF at different ages
        DX_grouped['Zscore_cumulative_citations_age'] = (DX_grouped['citation_count_age'].cumsum()-DX_grouped['EV_num_cit_total_age'].cumsum())/DX_grouped['STD_num_cit_total_age'].cumsum()
        Zcum0 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==0]
        Zcum1 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==1]
        Zcum2 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==2]
        Zcum3 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==3]
        Zcum4 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==4]
        Zcum5 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==5]
        Zcum10 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==10]
        Zcum20 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==20]
        Zcum30 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==30]
        Zcum40 = DX_grouped['Zscore_cumulative_citations_age'][DX_grouped['Age']==40]
        
        # compute cumulative sum of z scores of observed PDF at different ages
        DX_grouped['Zscore_citations_age_cumsumZ'] = DX_grouped['Zscore_citations_age'].cumsum()
        Zsum0 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==0]
        Zsum1 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==1]
        Zsum2 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==2]
        Zsum3 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==3]
        Zsum4 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==4]
        Zsum5 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==5]
        Zsum10 = DX_grouped['Zscore_citations_age_cumsumZ'][DX_grouped['Age']==10]
    
        # compute age corresponding to peak in z-score of PDF and last age in which Z-score of PDF is positive
        peakZ = DX_grouped['Age'][DX_grouped['Zscore_citations_age']==DX_grouped['Zscore_citations_age'].max()]
        first_posZ = DX_grouped['Age'][DX_grouped['Zscore_citations_age']>=0][:1]
        last_posZ = DX_grouped['Age'][DX_grouped['Zscore_citations_age']>=0][-1:]
        
        if not array(peakZ):
            peakZ = 'NaN'
        else: peakZ = peakZ

        if not array(first_posZ):
            first_posZ = 'NaN'
        else: first_posZ = first_posZ

        if not array(last_posZ):
            last_posZ = 'NaN'
        else: last_posZ = last_posZ
            
        # append yearly data to big dataframe
        DZyear = pd.DataFrame({'Domain':domain_name, 'year':year, 'Z0':array(Z0), 'Z1':array(Z1), 'Z2':array(Z2), 'Z3':array(Z3),
                               'Z4':array(Z4), 'Z5':array(Z5),'Z10':array(Z10), 'Z20':array(Z20), 'Z30':array(Z30), 'Z40':array(Z40),
                            'Zcum0':array(Zcum0), 'Zcum1':array(Zcum1), 'Zcum2':array(Zcum2), 'Z3cum':array(Zcum3), 'Zcum4':array(Zcum4), 
                           'Zcum5':array(Zcum5),'Zcum10':array(Zcum10), 'Zcum20':array(Zcum20), 'Zcum30':array(Zcum30), 'Zcum40':array(Zcum40),
                            'Zsum0':array(Zsum0), 'Zsum1':array(Zsum1), 'Zsum2':array(Zsum2), 'Zsum3':array(Zsum3), 'Zsum4':array(Zsum4), 
                              'Zsum5':array(Zsum5), 'Zsum10':array(Zsum10),
                               'peakZ':array(peakZ), 'first_posZ':array(first_posZ), 'last_posZ':array(last_posZ)})
        DZ_years_domains = DZ_years_domains.append(DZyear)  
    

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

In [ ]:
idx_notNaN = DZ_years_domains['last_posZ']!='NaN'
DZ_years_domains['Zcum5_over_last_ageZpos'] = 'NaN'
DZ_years_domains['Zcum5_over_last_ageZpos'][idx_notNaN] = DZ_years_domains['Zcum5'][idx_notNaN] / DZ_years_domains['last_posZ'][idx_notNaN]
DZ_years_domains['Zcum10_over_last_ageZpos'] = 'NaN'
DZ_years_domains['Zcum10_over_last_ageZpos'][idx_notNaN] = DZ_years_domains['Zcum10'][idx_notNaN] / DZ_years_domains['last_posZ'][idx_notNaN]

# Export DataFrame with Observed and Expected Values of citation count by age at patent level

In [ ]:
ti = time.time()
(DF_final.to_csv(export_path+'observed_vs_expected_citation_age_patent_level.csv',index=False))
 tf = time.time()
(tf-ti)/60

# Export DataFrame with Obsolescence Indices over time for each Domain

In [ ]:
ti = time.time()
DZ_years_domains.to_csv(export_path+'DataFrame_Normalized_Obsolescence_Domains_over_time.csv',index=False)
tf = time.time()
(tf-ti)/60

In [ ]:
time_end_notebook = time.time()
(time_end_notebook - time_start_notebook = time.time())/60